In [2]:
import cv2
import time
import mediapipe as mp

class handDetector:
    def __init__(self, mode=False, max_hands=2, modelComp=1, det_conf=0.5, track_conf=0.5):

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(mode, max_hands, modelComp, det_conf, track_conf)
        self.mpDraw = mp.solutions.drawing_utils

    def findHands(self, img, draw=True, w_width=640, h_height=480):
        img = cv2.resize(img, (w_width, h_height))
        img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(img_RGB)

        if self.results.multi_hand_landmarks:
            for handlms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(
                        img, handlms, self.mpHands.HAND_CONNECTIONS,                                           # We used img here as we need to draw on img not img_RGB
                        landmark_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(                          # Specifies drawing properties for landmarks
                            color=(255, 0, 0), thickness=4, circle_radius=2),                                  # color is taken as RGB (as we have converted to RGB)
                        connection_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(                        # Specifies drawing properties for connections
                            color=(0, 0, 255), thickness=2, circle_radius=2)                                   # Thickness and Circle_Radius to be int values
                    )
        return img

    def findPos(self, img, handNo=0, draw=True, w_width=640, h_height=480):
        lmlist = []
        if self.results.multi_hand_landmarks:
            handlms = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(handlms.landmark):
                cx, cy = int(lm.x * w_width), int(lm.y * h_height)
                lmlist.append([id, cx, cy])
                if id == 8 and draw:
                    cv2.circle(img, (cx, cy), 10, (255, 0, 255), 10)                # pass img, string to pe printed, location, font type, font size, font color & font thickn.
        return lmlist

def main():
    cap = cv2.VideoCapture(0)
    prev_time = 0
    curr_time = 0

    w_width = 1150
    h_height = 720
    detect = handDetector()
    while cap.isOpened():
        success, img = cap.read()
        if not success:
            print("Failed to capture image")
            break

        img = cv2.flip(img, 1)

        img = detect.findHands(img, True, w_width, h_height)
        lmlist = detect.findPos(img, 0,True,w_width , h_height)

        curr_time = time.time()
        fps = 1 / (curr_time - prev_time)
        prev_time = curr_time
        cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)

        cv2.imshow("Webcam", img)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
# Jab image par draw karna ho toh ya toh usko RGB me rakho and phir cv2.imshow ke time se pehle BGR me convert kardo ya phir seedha BGR image par hi draw kardo